In [3]:
import numpy as np
import pandas as pd

In [4]:
def get_data(comm_markets_path, distances_path):
    '''
    A function that pulls in the data from marketcommodities.ini and the flc dump file and scrapes it. 
    +++++++++
    Parameters:
    comm_markets_path (str): the path of market_commodities.ini
    distances_path (str): the path of the flc dump file
    +++++++++
    Returns
    distances (Dataframe): pandas dataframe of all the viable distances between bases
    bases(list of dictonaries): list containing all of the base entries in market_commodities.ini in dictonary 
    form. Dictonaies hold base_code, base_name, and list of commodities traded. 
    comm_set (set): set of all commodities bought or sold on all bases
    base_names (list): all base names
    '''


    distances = pd.read_csv(distances_path, names = ['start', 'end', 'time'])
    distances = distances[distances['start']!=distances['end']]
    distances = distances[distances['time']!=-1]

    with open(comm_markets_path, 'r') as file:
        lines = file.readlines() 
    
    comment_count = 0
    bases = []
    commodities =[]
    for i in lines[2:]:
        if i.lower() == '[basegood]\n' or i == ';EVERYTHING BELOW THIS LINE IS DATABASE.':
            if comment =='':
                comment = base_code
            bases.append({'base_code':base_code, 'Name':comment, 'commodities':commodities})
            commodities=[]
            comment = ''
            comment_count = 0
        if i[:4].lower() == "base":
            base_code = i[7:]
        if i[:1] == ';' and comment_count == 0:
            comment = i
            comment_count+=1
        elif i[:1]==';':
            print(i)
            comment +'\n'+ i
        if i[:10].lower() == 'marketgood':
            t =i[13:].split()
            l = [float(j.strip(',')) for j in t[1:6]]
            l.append(float(t.pop()))
            l.append(t[0])
            commodities.append(l)
    
    comm_set = set([])
    for i in bases: #this loop in loop adds all the commodities to a set so we know how many commodities to display in our applet
        j = i['commodities']
        for k in j:
            comm_set.add(k[6])
    base_names=[i['Name'].strip('\n') for i in bases]
    
    return distances, bases, comm_set, base_names,

In [5]:
def which_sell(commodity_list):
    sells_set = set([])
    for i in bases:
        for k in i['commodities']:
            if k[-1] in commodity_list and k[-3]== 0:
                sells_set.add(i['base_code'])
    l = list(sells_set)
    return [i.strip('\n') for i in l]

In [ ]:
def sort_by_closest_base(list_of_bases, distances):
    '''
    a function that takes the distances dataframe and any list of bases and sorts all the other bases out into the lists of bases closest to those bases in the list of bases. 
    Its got a lot of fuckery, and maybe can be given aother pass for better algorithmic complexity. I'm so sorry. 
    
    Parameters 
    list_of_bases (list): a list of string base codes
    distances (DataFrame): the base to base travel times from flcomp formatted as a DataFrame by get_data()
    
    Returns:
    sorted_l (list of lists) list of list of bases, sorted in the same shape as the list of bases fed in, 
    with each list containing those bases that are closest to the base listed at the same index in list_of_bases
    '''
    k = [distances[distances['start'] == i] for i in list_of_bases] # makes a list of dataframes of routes starting from the bases in question
    k = [i[~i['end'].isin(list_of_bases)].reset_index() for i in k] # gets rid of routes between bases in the  list
    
    df = pd.DataFrame()
    for idx, i in enumerate(list_of_bases): #peices the list k back together into a larger thing
        df[i]=k[idx]['time']
    df['base'] = k[0]['end']
    df = df.set_index('base')

    sorted_bases = df.idxmin(axis=1)
    sorted_bases = sorted_bases.reset_index()
    sorted_l =[]

    for i in list_of_bases:
        sorted_l.append(sorted_bases[sorted_bases[0]==i]['base'].to_list())

    return sorted_l

In [31]:
path_markets = 'market_commodities.ini'
distances_path = 'dump.csv'

In [32]:
distances, bases, comm_set, base_names = get_data(path_markets, distances_path)

In [126]:
now, suppose we sell a commodity from X bases. sort all the other bases into shortest distance from those bases. 
    
        

297

In [33]:
list_of_bases = ['li01_01_base','rh01_01_base','br01_01_base', 'ku01_01_base', 'ga01_01_base']

In [ ]:
drop the things in the list from end

put the dataframes back together, get rid of start and keep the first end (new index) and the times labled as start base

In [242]:
def sort_by_closest_base(list_of_bases, distances):
    '''
    a function that takes the distances dataframe and any list of bases and sorts all the other bases out into the lists of bases closest to those bases in the list of bases. 
    Its got a lot of fuckery, and maybe can be given aother pass for better algorithmic complexity. I'm so sorry. 
    
    Parameters 
    list_of_bases (list): a list of string base codes
    distances (DataFrame): the base to base travel times from flcomp formatted as a DataFrame by get_data()
    
    Returns:
    sorted_l (list of lists) list of list of bases, sorted in the same shape as the list of bases fed in, 
    with each list containing those bases that are closest to the base listed at the same index in list_of_bases
    '''
    k = [distances[distances['start'] == i] for i in list_of_bases] # makes a list of dataframes of routes starting from the bases in question
    k = [i[~i['end'].isin(list_of_bases)].reset_index() for i in k] # gets rid of routes between bases in the  list
    
    df = pd.DataFrame()
    for idx, i in enumerate(list_of_bases): #peices the list k back together into a larger thing
        df[i]=k[idx]['time']
    df['base'] = k[0]['end']
    df = df.set_index('base')

    sorted_bases = df.idxmin(axis=1)
    sorted_bases = sorted_bases.reset_index()
    sorted_l =[]

    for i in list_of_bases:
        sorted_l.append(sorted_bases[sorted_bases[0]==i]['base'].to_list())

    return sorted_l

In [243]:
l = sort_by_closest_base(list_of_bases, distances)

In [250]:
len(l[0])

94

In [149]:
b =[i['time'].tolist() for i in k]
b.append(k[0]['end'].tolist())

In [190]:
for i in g:
    print(i)

In [158]:
b =[i['time'].tolist() for i in k]
b.append(k[0]['end'].tolist())

b=np.array(b)

b.T

array([['613767', '822295', '8396', '930644', '971716', 'br01_04_base'],
       ['547640', '814798', '68277', '911424', '952496', 'br01_05_base'],
       ['611624', '830470', '7046', '920594', '961666', 'br01_06_base'],
       ...,
       ['1063804', '1060542', '1592134', '1138015', '1475946',
        'st39_02_base'],
       ['1238729', '1235467', '1767059', '1312940', '1650871',
        'st39_03_base'],
       ['1056118', '1052856', '1584448', '1130329', '1468260',
        'st39_04_base']], dtype='<U21')

In [159]:
k

[               start           end     time
 217669  li01_01_base  br01_04_base   613767
 217670  li01_01_base  br01_05_base   547640
 217671  li01_01_base  br01_06_base   611624
 217672  li01_01_base  br01_07_base   544251
 217673  li01_01_base  br01_08_base   684300
 ...              ...           ...      ...
 218224  li01_01_base  st08_02_base   914687
 218225  li01_01_base  st39_01_base  1139393
 218226  li01_01_base  st39_02_base  1063804
 218227  li01_01_base  st39_03_base  1238729
 218228  li01_01_base  st39_04_base  1056118
 
 [549 rows x 3 columns],                start           end     time
 267598  rh01_01_base  br01_04_base   822295
 267599  rh01_01_base  br01_05_base   814798
 267600  rh01_01_base  br01_06_base   830470
 267601  rh01_01_base  br01_07_base   821598
 267602  rh01_01_base  br01_08_base   959215
 ...              ...           ...      ...
 268153  rh01_01_base  st08_02_base   911425
 268154  rh01_01_base  st39_01_base  1136131
 268155  rh01_01_base  st39_0

In [94]:
distances_comb = k[0][['end', 'time']].rename_axis('time', list_of_bases[0])


TypeError: rename_axis() takes from 1 to 2 positional arguments but 3 were given

In [82]:
from sklearn.cluster import KMeans

In [90]:
k[4][distances['end'] !=any(list_of_bases)]

/home/robert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,start,end,time
110518,ga01_01_base,br01_04_base,971716
110519,ga01_01_base,br01_05_base,952496
110520,ga01_01_base,br01_06_base,961666
110521,ga01_01_base,br01_07_base,947671
110522,ga01_01_base,br01_08_base,950393
...,...,...,...
111073,ga01_01_base,st08_02_base,1326257
111074,ga01_01_base,st39_01_base,1551535
111075,ga01_01_base,st39_02_base,1475946
111076,ga01_01_base,st39_03_base,1650871


In [58]:
food = ['commodity_food,',
'commodity_food_br,',
'commodity_food_ga,',
'commodity_luxury_food,',
'commodity_rice,',
'commodity_gaianlife,',
'commodity_synthpaste,',]

In [14]:
len(bases)

604

In [15]:
distances

,start,end,time
1,li01_11_base,li01_01_base,57793
2,li01_11_base,li01_03_base,60731
3,li01_11_base,li01_09_base,106869
4,li01_11_base,li01_12_base,180401
5,li01_11_base,li01_10_base,126973
...,...,...,...
27882,ew06_02_base,ew04_01_base,275520
27883,ew06_02_base,hi01_01_base,502934
27884,ew06_02_base,hi02_01_base,273235
27885,ew06_02_base,hi02_02_base,322338


In [20]:
k[1]

,start,end,time
14863,rh01_01_base,li01_11_base,646454
14864,rh01_01_base,li01_01_base,698582
14865,rh01_01_base,li01_03_base,587719
14866,rh01_01_base,li01_09_base,651402
14867,rh01_01_base,li01_12_base,651348
...,...,...,...
15024,rh01_01_base,hi01_01_base,675462
15025,rh01_01_base,hi02_01_base,742449
15026,rh01_01_base,hi02_02_base,704018
15028,rh01_01_base,ew06_01_base,772501


In [22]:
len(bases)

604

In [28]:
distances = pd.read_csv(distances_path, names = ['start', 'end', 'time'])
#     distances = distances[distances['start']!=distances['end']]
#     distances = distances[distances['time']!=-1]

In [29]:
distances

,start,end,time
0,li01_11_base,li01_11_base,0
1,li01_11_base,li01_01_base,57793
2,li01_11_base,li01_03_base,60731
3,li01_11_base,li01_09_base,106869
4,li01_11_base,li01_12_base,180401
...,...,...,...
27884,ew06_02_base,hi02_01_base,273235
27885,ew06_02_base,hi02_02_base,322338
27886,ew06_02_base,st01_01_base,-1
27887,ew06_02_base,ew06_01_base,96435


In [30]:
604*604

364816